In [1]:
from pathlib import Path
import os

In [19]:
base_path = Path("../data/train_layouts_2")
# Директории
train_img_dir = base_path / "train" / "images"
train_lbl_dir = base_path / "train" / "labels"

valid_img_dir = base_path / "valid" / "images"
valid_lbl_dir = base_path / "valid" / "labels"

test_img_dir = base_path / "test" / "images"
test_lbl_dir = base_path / "test" / "labels"

yaml_path = base_path / "data.yaml" 

In [20]:
def remove_class_from_labels(label_dir, class_ids_to_remove):
    for filename in os.listdir(label_dir):
        if filename.endswith('.txt'):
            path = os.path.join(label_dir, filename)
            with open(path, 'r') as file:
                lines = file.readlines()
            # Удалим строки, начинающиеся с любого class_id из списка
            new_lines = [
                line for line in lines 
                if not any(line.startswith(f"{class_id} ") for class_id in class_ids_to_remove)
            ]
            with open(path, 'w') as file:
                file.writelines(new_lines)


In [21]:
# Применим для train и valid, test
remove_class_from_labels(train_lbl_dir, [0,2,3])
remove_class_from_labels(valid_lbl_dir, [0,2,3])
remove_class_from_labels(test_lbl_dir, [0,2,3])

In [22]:
# Папки с аннотациями
label_dirs = [
    Path(train_lbl_dir),
    Path(valid_lbl_dir),
    Path(test_lbl_dir)
]

# Перенумерация классов: старая -> новая
id_map = {1: 0,
          4: 1}

for label_dir in label_dirs:
    print(f"\n🗂 Обработка папки: {label_dir}")

    for label_file in label_dir.glob("*.txt"):
        changed = False
        new_lines = []

        with open(label_file, "r") as f:
            lines = f.readlines()

        for line_num, line in enumerate(lines, 1):
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"⚠️  Пропущена строка с ошибкой в {label_file.name}, строка {line_num}: {line.strip()}")
                continue

            try:
                old_id = int(parts[0])
            except ValueError:
                print(f"❌ Невозможно преобразовать class_id в число: {parts[0]}")
                continue

            if old_id in id_map:
                parts[0] = str(id_map[old_id])
                changed = True
                new_lines.append(" ".join(parts))
            else:
                print(f"❌ Неизвестный class_id {old_id} в {label_file.name}, строка {line_num}")
                continue

        if changed:
            with open(label_file, "w") as f:
                f.write("\n".join(new_lines) + "\n")
            print(f"✅ Обновлён: {label_file.name}")
        else:
            print(f"ℹ️ Без изменений: {label_file.name}")




🗂 Обработка папки: ../data/train_layouts_2/train/labels
✅ Обновлён: 4074_png.rf.0d1afac1acccc40398f8807595add121.txt
✅ Обновлён: 7_png.rf.2680f09a40f0ad6f12df025be6f70dbb.txt
✅ Обновлён: 4051_png.rf.2406eca4e205694e7b5670c5efbe3862.txt
✅ Обновлён: 62_png.rf.c7ee6f29bfcd6c6a1749c809b72065e0.txt
ℹ️ Без изменений: 142_png.rf.2e3f4fbec8acfb633ecb168ad990cd0f.txt
✅ Обновлён: 14_png.rf.dbc79ec9c335cc5a27265a3c13ff19fd.txt
✅ Обновлён: 4083_png.rf.62faf7b2f00f4d815436422b86ca6e28.txt
ℹ️ Без изменений: 142_png.rf.ab8c9fd8f7030df015904b2bf1516d5c.txt
✅ Обновлён: 61_png.rf.d660826ba72a01a6f2ba1767b60b9d8e.txt
✅ Обновлён: 145_png.rf.d196193cd69ab55bbf3700cbf40a0c08.txt
✅ Обновлён: 4097_png.rf.31d7d18023438c2410e7046d5cde8646.txt
✅ Обновлён: 4076_png.rf.8b42cb7c728c19b766f23f24e806c881.txt
✅ Обновлён: 207_png.rf.106f6a64b26606f041c2bef1f8676815.txt
✅ Обновлён: 27_png.rf.0cd849281c8f6459db4be26b16162c11.txt
✅ Обновлён: 4053_png.rf.35d6807c577209efa181a02ff6a40bb3.txt
✅ Обновлён: 185_png.rf.a24654b2

Если остались файлы без нужной разметки, их можно удалить.

In [6]:
# Поддерживаемые расширения изображений
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

def delete_empty(labels_dir, images_dir):
    print("Обрабатываю директорию: ", labels_dir, images_dir)
    # Поддерживаемые расширения изображений
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

    # Список удалённых пар (label, image)
    removed_files = []

    # Проходим по всем файлам в папке labels
    for label_file in os.listdir(labels_dir):
        if label_file.endswith(".txt"):
            label_path = os.path.join(labels_dir, label_file)
            
            # Если файл пустой
            if os.path.getsize(label_path) == 0:
                # Имя без расширения
                filename_wo_ext = os.path.splitext(label_file)[0]
                print('удаляем разметку:', filename_wo_ext)
                # Удаляем пустой txt
                os.remove(label_path)

                # Пытаемся найти и удалить соответствующее изображение
                image_deleted = False
                for ext in image_extensions:
                    image_path = os.path.join(images_dir, filename_wo_ext + ext)
                    if os.path.exists(image_path):
                        print('удаляем изображение:', image_path)
                        os.remove(image_path)
                        removed_files.append((label_file, filename_wo_ext + ext))
                        image_deleted = True
    print('всего удалено:', len(removed_files))
    removed_files


In [23]:
delete_empty(train_lbl_dir, train_img_dir)

Обрабатываю директорию:  ../data/train_layouts_2/train/labels ../data/train_layouts_2/train/images
удаляем разметку: 142_png.rf.2e3f4fbec8acfb633ecb168ad990cd0f
удаляем изображение: ../data/train_layouts_2/train/images/142_png.rf.2e3f4fbec8acfb633ecb168ad990cd0f.jpg
удаляем разметку: 142_png.rf.ab8c9fd8f7030df015904b2bf1516d5c
удаляем изображение: ../data/train_layouts_2/train/images/142_png.rf.ab8c9fd8f7030df015904b2bf1516d5c.jpg
удаляем разметку: 195_png.rf.336154fa6f559b8542b4c2faaca3c173
удаляем изображение: ../data/train_layouts_2/train/images/195_png.rf.336154fa6f559b8542b4c2faaca3c173.jpg
удаляем разметку: 14_png.rf.ef120143bfb5de4aca1c86e7753471fc
удаляем изображение: ../data/train_layouts_2/train/images/14_png.rf.ef120143bfb5de4aca1c86e7753471fc.jpg
удаляем разметку: 146_png.rf.c2ef8dc3739b65677eec1639ab66791d
удаляем изображение: ../data/train_layouts_2/train/images/146_png.rf.c2ef8dc3739b65677eec1639ab66791d.jpg
удаляем разметку: 195_png.rf.20921ed13ab3fc28ea9d2ebd09706216
у

In [24]:
delete_empty(valid_lbl_dir, valid_img_dir)

Обрабатываю директорию:  ../data/train_layouts_2/valid/labels ../data/train_layouts_2/valid/images
всего удалено: 0


In [25]:
delete_empty(test_lbl_dir, test_img_dir)

Обрабатываю директорию:  ../data/train_layouts_2/test/labels ../data/train_layouts_2/test/images
всего удалено: 0
